In [128]:
from ast import literal_eval
from bs4 import BeautifulSoup
from requests import get

def macdonald(page):
    '''
    맥도날드 페이지에서, 메뉴들을 크롤링 해온다.
    '''
    
    #url, header 설정
    url = "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=2&catId={}".format(page)
    u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"
    
    #응답 값을 받아오고, bs 객체로 변환한다.
    response = requests.get(url, headers={"USER-AGENT":u_a})
    soup = BeautifulSoup(response.text, 'html.parser')
        
    #상품부분에 해당하는 부분을 find, find_all로 필터링 한다.
    products_box_soup = soup.find("div", {"id":"product-cards"})
    products = products_box_soup.find_all("a", {"class": "btn btn-block action-create btn-yellow"})

    #필터링 된 정보를 재가공 한다.
    informations = []
    for product in products:
        data = product.get('onclick')
        if data.startswith('onProductClick'):
            informations.append(data[len('onProductClick('): -1])
    
    result = []
    for information in informations:
        temp_dict = {}
        menu = literal_eval(information)
        temp_dict['name'] = menu['name']
        temp_dict['price'] = menu['price'][2:]
        temp_dict['category'] = menu['cat']
        result.append(temp_dict)
        
    #결과를 DB에 삽입한다.
    for a in result:
        print(a)
    print("\n\n")
    
    
macdonald(10)
macdonald(11)
macdonald(13)
macdonald(14)
macdonald(15)
macdonald(16)
    

{'name': '딸기 칠러', 'price': '3,100', 'category': '추천 메뉴'}
{'name': '콘 스프', 'price': '2,100', 'category': '추천 메뉴'}
{'name': '베이컨 에그 맥머핀™', 'price': '4,600', 'category': '추천 메뉴'}
{'name': '상하이 치킨 스낵랩', 'price': '3,900', 'category': '추천 메뉴'}
{'name': '골든 모짜렐라 치즈스틱 2조각', 'price': '2,600', 'category': '추천 메뉴'}



{'name': '상하이 치킨 스낵랩', 'price': '2,600', 'category': '맥모닝 & 세트'}
{'name': '에그 맥머핀™', 'price': '3,100', 'category': '맥모닝 & 세트'}
{'name': '베이컨 에그 맥머핀™', 'price': '3,500', 'category': '맥모닝 & 세트'}
{'name': '베이컨 토마토 머핀', 'price': '3,600', 'category': '맥모닝 & 세트'}
{'name': '소시지 에그 맥머핀™', 'price': '3,500', 'category': '맥모닝 & 세트'}
{'name': '소시지 맥머핀™', 'price': '2,600', 'category': '맥모닝 & 세트'}
{'name': '디럭스 브렉퍼스트', 'price': '5,200', 'category': '맥모닝 & 세트'}
{'name': '핫케익 2조각', 'price': '2,600', 'category': '맥모닝 & 세트'}
{'name': '핫케익 3조각', 'price': '3,400', 'category': '맥모닝 & 세트'}
{'name': '딸기잼', 'price': '200', 'category': '맥모닝 & 세트'}
{'name': '핫케익 시럽 추가', 'price': '300', 'category': '맥모닝 & 세트'

In [53]:
from ast import literal_eval
from bs4 import BeautifulSoup
import requests


def momstouch(menu):
    '''
    맘스터치 페이지에서, 메뉴들을 크롤링 해온다.
    '''
    
    result = []
    menu_page = []
    for page in range(1,100):
        #url, header 설정
        url = "http://www.momstouch.co.kr/sub/menu/menu_list.html?pg={}&menu={}".format(page,menu)
        u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"

        #응답 값을 받아오고, bs 객체로 변환한다.
        response = requests.get(url, headers={"USER-AGENT":u_a})
        soup = BeautifulSoup(response.content.decode('euc-kr','replace'))
        
        if "메뉴 준비중입니다" in soup.text:
            break

        products_box_soup = soup.find("div", {"class": "menu_list"})
        products = products_box_soup.find_all("img")
        title = soup.find("strong")

        site_src = "http://www.momstouch.co.kr/"

        
        for product in products:
            temp_dict = {}
            data = product.get('alt')
            resource = product.get("src")
            if data is not "":
                temp_dict['name'] = data
                temp_dict['category'] = title.text.strip()
                temp_dict['image'] = site_src + resource.replace("../", "")
                menu_page.append(temp_dict)
                
    return menu_page
        
for k in [1,4,5,6]:
    for j in momstouch(k):
        print(j)
    print("\n\n")
#momstouch(1)
#momstouch(4)
#momstouch(5)
#momstouch(6)
    

{'name': '마살라 치킨', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180704483349.jpg'}
{'name': '마살라 텐더', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180704993362.jpg'}
{'name': '마살라 윙', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180704877799.jpg'}
{'name': '찜햇닭', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20171205680678.jpg'}
{'name': '찜햇닭 강정', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20171205956642.jpg'}
{'name': '찜햇닭 윙', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20171205654760.jpg'}
{'name': '치파오치킨', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20170726800865.jpg'}
{'name': '치파오', 'category': '치킨 메뉴', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20170726106661.jpg'}
{

{'name': '레몬홍차', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20181113571936.jpg'}
{'name': '홍차', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20181113609096.jpg'}
{'name': '복숭아망고스무디(홈카페 전용)', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180806337149.jpg'}
{'name': '레몬딸기스무디(홈카페 전용)', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180806805736.jpg'}
{'name': '딸기크림컵빙수', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180416547805.jpg'}
{'name': '딸기크림빙수', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180416594022.jpg'}
{'name': '레몬에이드', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180220239529.jpg'}
{'name': '레몬티', 'category': '음료류', 'image': 'http://www.momstouch.co.kr/data/shopimages/xboard/menu/20180220231170.jpg'

In [58]:
import requests

from ast import literal_eval
from bs4 import BeautifulSoup
from requests import get

def subway():
    '''
    서브웨이 페이지에서, 메뉴들을 크롤링 해온다.
    '''
    
    #url, header 설정
    url = "http://subway.co.kr/sandwichList"
    u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"
    
    
    
    #응답 값을 받아오고, bs 객체로 변환한다.
    response = requests.get(url, headers={"USER-AGENT":u_a})
    soup = BeautifulSoup(response.text, 'html.parser')
    

    category = {
        'new': '신제품', 
        'promotion': '프로모션',
        'cl': '클래식',
        'fl': '프레쉬&라이트',
        'pl': '프리미엄',
        'bf': '아침메뉴',
    }
    
    result = []
    for k, v in category.items():
        products = soup.find_all("li" , {'class':'{}'.format(k)})
        
        for product in products:
            image = product.img.get('src')
            name = product.img.get('alt')
            
            temp_dict = {}
            temp_dict['name'] = name
            temp_dict['price'] = ""
            temp_dict['category'] = v
            temp_dict['image'] = image.replace("..","http://subway.co.kr")
            result.append(temp_dict)
            
    
    for a in result:
        print(a)
        

subway()

{'name': '쉬림프', 'price': '', 'category': '신제품', 'image': 'http://subway.co.kr/images/menu/sandwich_pm10.jpg'}
{'name': '쉬림프 아보카도', 'price': '', 'category': '신제품', 'image': 'http://subway.co.kr/images/menu/sandwich_pm11.jpg'}
{'name': '에그마요', 'price': '', 'category': '클래식', 'image': 'http://subway.co.kr/images/menu/sandwich_cl06.jpg'}
{'name': '이탈리안 비엠티', 'price': '', 'category': '클래식', 'image': 'http://subway.co.kr/images/menu/sandwich_cl01.jpg'}
{'name': '비엘티', 'price': '', 'category': '클래식', 'image': 'http://subway.co.kr/images/menu/sandwich_cl02.jpg'}
{'name': '미트볼', 'price': '', 'category': '클래식', 'image': 'http://subway.co.kr/images/menu/sandwich_cl03.jpg'}
{'name': '햄', 'price': '', 'category': '클래식', 'image': 'http://subway.co.kr/images/menu/sandwich_cl04.jpg'}
{'name': '참치', 'price': '', 'category': '클래식', 'image': 'http://subway.co.kr/images/menu/sandwich_cl05.jpg'}
{'name': '로티세리 치킨', 'price': '', 'category': '프레쉬&라이트', 'image': 'http://subway.co.kr/images/menu/sandwich_fl01.

In [66]:
import requests

from ast import literal_eval
from bs4 import BeautifulSoup
from requests import get

def popeyes(page):
    '''
    popeyes 페이지에서, 메뉴들을 크롤링 해온다.
    '''
    
    #url, header 설정
    url = "http://www.popeyes.co.kr/menu/list.asp?cate={}".format(page)
    u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"
    
    
    
    #응답 값을 받아오고, bs 객체로 변환한다.
    response = requests.get(url, headers={"USER-AGENT":u_a})
    soup = BeautifulSoup(response.text, 'html.parser')
    
    cate = {
        96: '치킨',
        97: '치킨텐더',
        98: '테이크아웃팩',
        99: '버거',
        100: '사이드',
        101: '음료',
        102: '소스',
    }
    

    
    products = soup.find_all("div", {'class': 'flavor-details'})
    
    result = []
    for product in products:
        image = product.img.get("src")
        name = product.img.get("alt")
        category = cate[page]
        
        temp_dict = {}
        temp_dict['image'] = image
        temp_dict['name'] = name
        temp_dict['category'] = category
        temp_dict['price'] = ""
        

        
        result.append(temp_dict)
    
    for a in result:
        print(a)

    
#     result = []
#     for k, v in category.items():
#         products = soup.find_all("li" , {'class':'{}'.format(k)})
        
#         for product in products:
#             image = product.img.get('src')
#             name = product.img.get('alt')
            
#             temp_dict = {}
#             temp_dict['name'] = name
#             temp_dict['price'] = ""
#             temp_dict['category'] = v
#             temp_dict['image'] = image.replace("..","http://subway.co.kr")
#             result.append(temp_dict)
            
    
#     for a in result:
#         print(a)
        

popeyes(100)

{'image': 'http://www.popeyes.co.kr/images/goods/small/side_01.png', 'name': '케이준후라이 R', 'category': '사이드', 'price': ''}
{'image': 'http://www.popeyes.co.kr/images/goods/small/06_02.png', 'name': '케이준후라이 L', 'category': '사이드', 'price': ''}
{'image': 'http://www.popeyes.co.kr/images/goods/small/78.png', 'name': '비스킷', 'category': '사이드', 'price': ''}
{'image': 'http://www.popeyes.co.kr/images/goods/small/06_09.png', 'name': '코올슬로', 'category': '사이드', 'price': ''}
{'image': 'http://www.popeyes.co.kr/images/goods/small/06_03.png', 'name': '콘샐러드', 'category': '사이드', 'price': ''}
{'image': 'http://www.popeyes.co.kr/images/goods/small/06_04.png', 'name': '치즈스틱', 'category': '사이드', 'price': ''}
